In [2]:
pip install -U bitsandbytes>=0.46.1

In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, pipeline
from huggingface_hub import login, notebook_login
from IPython.display import display, Markdown

In [4]:
def print_markdown(text: str):
  return display(Markdown(text))
model_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:

# quantization settings
quantization_configs = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_compute_dtype=torch.bfloat16,
  bnb_4bit_quant_type='nf4',
  bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
  model_id,
  attn_implementation="eager",
  quantization_config=quantization_configs,
  device_map="auto",
  trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

prompts = [
  "if (5x - 10) = 2, solve for x",
  "if x = [1 0 2] and y = [0 1 0] find x^T y",
  r"""
**Advanced Linear Algebra — Singular Value Decomposition (SVD) Computation**
Let
\[
A =
\begin{bmatrix}
4 & 0 & 2 \\
1 & 3 & 1 \\
0 & 2 & 2
\end{bmatrix}.
\]

Compute the full Singular Value Decomposition of \(A\). That is, determine orthogonal matrices \(U \in \mathbb{R}^{3 \times 3}\) and \(V \in \mathbb{R}^{3 \times 3}\), along with a diagonal matrix \(\Sigma \in \mathbb{R}^{3 \times 3}\) with non-negative entries, such that

\[
A = U \Sigma V^\top.
\]

Your computation must proceed from first principles: derive the singular values via the eigenvalues of \(A^\top A\), construct \(V\) from the corresponding orthonormal eigenvectors, compute \(U\) using \(U = A V \Sigma^{-1}\), and explicitly verify the decomposition by matrix multiplication. Finally, use the SVD to determine the rank and 2-norm condition number of \(A\).
"""
]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [6]:
inputs = [tokenizer(prompt, return_tensors='pt').to('cuda') for prompt in prompts]
outputs = model.generate(**inputs[-1], max_new_tokens=1500, streamer=streamer, return_dict_in_generate=True)
response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

# parse chain-of-thought tokens

def process_response(response):
  reason_start = response.find('<think>') + len('<think>')
  reason_end = response.find('</think>')
  reason = response[reason_start:reason_end].strip()

  # output
  output_start = reason_end + len('</think>')
  output = response[output_start:].strip()

  return reason, output

reason, output = process_response(response)
print_markdown(f"**Reasoning**: {reason}")
print_markdown(f"**Output**: {output}")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


</think>

Let

\[
A =
\begin{bmatrix}
4 & 0 & 2 \\
1 & 3 & 1 \\
0 & 2 & 2
\end{bmatrix}.
\]

Our goal is to compute the full Singular Value Decomposition (SVD) of \(A\), which involves finding orthogonal matrices \(U\) and \(V\), and a diagonal matrix \(\Sigma\), such that:

\[
A = U \Sigma V^\top.
\]

### Step 1: Compute \(A^\top A\)

First, we compute the product \(A^\top A\):

\[
A^\top A = \begin{bmatrix}
4 & 1 & 0 \\
0 & 3 & 2 \\
2 & 1 & 2
\end{bmatrix}
\begin{bmatrix}
4 & 0 & 2 \\
1 & 3 & 1 \\
2 & 1 & 2
\end{bmatrix}
= \begin{bmatrix}
21 & 11 & 1 \\
11 & 11 & 1 \\
5 & 5 & 5
\end{bmatrix}.
\]

### Step 2: Find the Eigenvalues of \(A^\top A\)

Next, we find the eigenvalues of the matrix \(A^\top A\). The eigenvalues \(\lambda_i\) satisfy the characteristic equation:

\[
\det(A^\top A - \lambda I) = 0.
\]

Solving this, we find the eigenvalues:

\[
\lambda_1 = 21, \quad \lambda_2 = 10, \quad \lambda_3 = 5.
\]

### Step 3: Compute the Singular Values of \(A\)

The singular values \(\

**Reasoning**: anced Linear Algebra — Singular Value Decomposition (SVD) Computation**
Let  
\[
A =
\begin{bmatrix}
4 & 0 & 2 \\
1 & 3 & 1 \\
0 & 2 & 2
\end{bmatrix}.
\]

Compute the full Singular Value Decomposition of \(A\). That is, determine orthogonal matrices \(U \in \mathbb{R}^{3 \times 3}\) and \(V \in \mathbb{R}^{3 \times 3}\), along with a diagonal matrix \(\Sigma \in \mathbb{R}^{3 \times 3}\) with non-negative entries, such that

\[
A = U \Sigma V^\top.
\]

Your computation must proceed from first principles: derive the singular values via the eigenvalues of \(A^\top A\), construct \(V\) from the corresponding orthonormal eigenvectors, compute \(U\) using \(U = A V \Sigma^{-1}\), and explicitly verify the decomposition by matrix multiplication. Finally, use the SVD to determine the rank and 2-norm condition number of \(A\).

**Output**: Let

\[
A =
\begin{bmatrix}
4 & 0 & 2 \\
1 & 3 & 1 \\
0 & 2 & 2
\end{bmatrix}.
\]

Our goal is to compute the full Singular Value Decomposition (SVD) of \(A\), which involves finding orthogonal matrices \(U\) and \(V\), and a diagonal matrix \(\Sigma\), such that:

\[
A = U \Sigma V^\top.
\]

### Step 1: Compute \(A^\top A\)

First, we compute the product \(A^\top A\):

\[
A^\top A = \begin{bmatrix}
4 & 1 & 0 \\
0 & 3 & 2 \\
2 & 1 & 2
\end{bmatrix}
\begin{bmatrix}
4 & 0 & 2 \\
1 & 3 & 1 \\
2 & 1 & 2
\end{bmatrix}
= \begin{bmatrix}
21 & 11 & 1 \\
11 & 11 & 1 \\
5 & 5 & 5
\end{bmatrix}.
\]

### Step 2: Find the Eigenvalues of \(A^\top A\)

Next, we find the eigenvalues of the matrix \(A^\top A\). The eigenvalues \(\lambda_i\) satisfy the characteristic equation:

\[
\det(A^\top A - \lambda I) = 0.
\]

Solving this, we find the eigenvalues:

\[
\lambda_1 = 21, \quad \lambda_2 = 10, \quad \lambda_3 = 5.
\]

### Step 3: Compute the Singular Values of \(A\)

The singular values \(\sigma_i\) are the square roots of the eigenvalues of \(A^\top A\):

\[
\sigma_1 = \sqrt{21}, \quad \sigma_2 = \sqrt{0} = 0, \quad \sigma_3 = \sqrt{5}.
\]

### Step 4: Compute the Matrix \(V\)

The matrix \(V\) is composed of the orthonormal eigenvectors of \(A^\top A\). We need to find these eigenvectors and orthogonalize them. However, due to the complexity of finding the eigenvectors manually, we will assume that \(V\) is orthogonal and proceed to compute \(U\) using the relation \(U = A V \Sigma^{-1}\).

### Step 5: Compute \(U\) using \(U = A V \Sigma^{-1}\)

Using the singular values and the matrix \(V\), we compute \(U\) as:

\[
U = A V \Sigma^{-1}.
\]

### Step6: Verify the SVD by Matrix Multiplication

Finally, we verify the SVD by computing \(U \Sigma V^\top\) to ensure it equals \(A\).

### Final Answer

The full Singular Value Decomposition of \(A\) is given by:

\[
U = \begin{bmatrix}
\sigma_1 & 0 & 0 \\
\sigma_2 & 0 & 0 \\
sigma_3 & 0 & 0
\end{bmatrix},
\]

\[
\Sigma = \begin{bmatrix}
\sqrt{21} & 0 & 0 \\
0 & 0 & 0 \\
0 & 0 & \sqrt{5}
\end{bmatrix},
\]

\[
V = \begin{bmatrix}
v_1 & v_2 & v_3 \\
v_4 & v_5 & v_6 \\
v_7 & v_8 & v_9
\end{bmatrix}.
\]

\end{think}

### Let's fine-tune Deepseek-R1-Qwen 1.5 b on math & physics problems

Datasets:

<b>Math</b>
```{python}
from datasets import load_dataset

ds = load_dataset("math-ai/StackMathQA", "stackmathqa1600k") # or any valid config_name
```

<b> General 'science' datasets </b>
<li> Scibench </li>
<li> Others </li>

```{python}
from datasets import load_dataset

dataset_names = [('xw27/scibench', 'xw27/scibench'), (name, config)]
for ds_name, config_name in dataset_names:
  ds = load_dataset(ds_name, config_name)
```


In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load datasets
print("Loading mathllm/MathCodeInstruct...")
math_dataset = load_dataset("mathllm/MathCodeInstruct", split="train")

print("Loading xw27/scibench...")
scibench_dataset = load_dataset("xw27/scibench", split="train")

# Display dataset sizes
print(f"\nOriginal dataset sizes:")
print(f"  MathCodeInstruct: {len(math_dataset):,} examples")
print(f"  SciBench: {len(scibench_dataset):,} examples")

# Calculate sampling to get ~25k total examples
# Allocate 60% to MathCodeInstruct (for tool-calling) and 40% to SciBench (for science)
max_total = 25000
math_samples = min(len(math_dataset), int(max_total * 0.6))  # 15k for tool-calling & math
scibench_samples = min(len(scibench_dataset), max_total - math_samples)  # ~10k for science

# Sample from datasets
print(f"\nSampling datasets to reach ~{max_total:,} examples...")
print(f"  Target: {math_samples:,} from MathCodeInstruct (tool-calling & math)")
print(f"  Target: {scibench_samples:,} from SciBench (science)")

math_subset = math_dataset.shuffle(seed=42).select(range(math_samples))
scibench_subset = scibench_dataset.shuffle(seed=42).select(range(scibench_samples))

# Combine datasets
print("\nCombining datasets...")
combined_dataset = concatenate_datasets([math_subset, scibench_subset])

# Shuffle the combined dataset
combined_dataset = combined_dataset.shuffle(seed=42)

print(f"\n✓ Combined dataset created:")
print(f"  Total examples: {len(combined_dataset):,}")
print(f"  - From MathCodeInstruct (tool-calling & math): {math_samples:,} ({math_samples/len(combined_dataset)*100:.1f}%)")
print(f"  - From SciBench (science): {scibench_samples:,} ({scibench_samples/len(combined_dataset)*100:.1f}%)")
print(f"\nDataset features: {combined_dataset.features}")

# Fine-Tuning Setup

Now we'll fine-tune DeepSeek-R1-Distill-Qwen-1.5B on our mixed math/science dataset using QLoRA.

In [ ]:
# Install fine-tuning dependencies
!pip install -q -U \
    transformers>=4.50.0 \
    peft>=0.14.0 \
    trl>=0.12.0 \
    accelerate>=1.2.0 \
    scipy \
    tensorboard

In [ ]:
# Format MathCodeInstruct to chat format with <think> tags
def format_mathcodeinstruct(example):
    """Convert MathCodeInstruct to DeepSeek-R1 chat format with reasoning."""
    messages = example.get('messages', [])
    if not messages:
        return None
    
    # Extract user question and assistant response
    user_msg = None
    assistant_msg = None
    
    for msg in messages:
        if msg.get('role') == 'user':
            user_msg = msg.get('content', '')
        elif msg.get('role') == 'assistant':
            assistant_msg = msg.get('content', '')
    
    if not user_msg or not assistant_msg:
        return None
    
    # Wrap assistant response with <think> tags for reasoning
    # MathCodeInstruct often includes code blocks which should be in reasoning
    formatted_response = f"<think>\n{assistant_msg}\n</think>\n\nThe solution is provided above."
    
    return {
        "messages": [
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": formatted_response}
        ]
    }

# Format SciBench to chat format with <think> tags
def format_scibench(example):
    """Convert SciBench to DeepSeek-R1 chat format with reasoning."""
    problem = example.get('problem_text', '')
    solution = example.get('solution', '')
    answer = example.get('answer_latex', '')
    unit = example.get('unit', '')
    
    if not problem or not solution:
        return None
    
    # Build final answer
    final_answer = answer
    if unit:
        final_answer = f"{answer} {unit}"
    
    # Format with reasoning in <think> tags
    formatted_response = f"<think>\n{solution}\n</think>\n\nFinal Answer: {final_answer}"
    
    return {
        "messages": [
            {"role": "user", "content": problem},
            {"role": "assistant", "content": formatted_response}
        ]
    }

print("Formatting datasets to chat format with <think> reasoning tags...")

# Apply formatting - need to determine which examples are from which dataset
# Since we concatenated, we'll create a source indicator
math_formatted = math_subset.map(
    format_mathcodeinstruct,
    remove_columns=math_subset.column_names,
    desc="Formatting MathCodeInstruct"
)

scibench_formatted = scibench_subset.map(
    format_scibench,
    remove_columns=scibench_subset.column_names,
    desc="Formatting SciBench"
)

# Filter out None values
math_formatted = math_formatted.filter(lambda x: x is not None and 'messages' in x)
scibench_formatted = scibench_formatted.filter(lambda x: x is not None and 'messages' in x)

# Recombine and shuffle
from datasets import concatenate_datasets
formatted_dataset = concatenate_datasets([math_formatted, scibench_formatted])
formatted_dataset = formatted_dataset.shuffle(seed=42)

print(f"\n✓ Formatted dataset: {len(formatted_dataset):,} examples")
print("\nExample:")
print(formatted_dataset[0])

In [ ]:
# Create 90/10 train/eval split
print("Creating train/eval split (90/10)...")
split_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print(f"\n✓ Split complete:")
print(f"  Training examples: {len(train_dataset):,}")
print(f"  Evaluation examples: {len(eval_dataset):,}")

In [ ]:
# Load model for fine-tuning with QLoRA
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

print("Loading model for fine-tuning...")

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager",  # Use eager for compatibility
)

# Prepare model for k-bit training
base_model = prepare_model_for_kbit_training(base_model)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Important for training

print("✓ Model and tokenizer loaded")

In [ ]:
# Configure LoRA
from peft import LoraConfig, get_peft_model

print("Configuring LoRA adapters...")

lora_config = LoraConfig(
    r=16,  # Rank - higher for complex reasoning tasks
    lora_alpha=32,  # Alpha scaling factor (2x rank is standard)
    target_modules="all-linear",  # Apply to all linear layers
    lora_dropout=0.05,  # Low dropout for instruction tuning
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to model
model = get_peft_model(base_model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"\n✓ LoRA configuration complete:")
print(f"  Trainable parameters: {trainable_params:,} ({trainable_percent:.2f}%)")
print(f"  Total parameters: {all_params:,}")
print(f"  LoRA rank: {lora_config.r}")
print(f"  LoRA alpha: {lora_config.lora_alpha}")

In [ ]:
# Training configuration
from transformers import TrainingArguments

output_dir = "./deepseek-r1-math-science"

training_args = TrainingArguments(
    output_dir=output_dir,
    
    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # Effective batch size = 2 * 8 = 16
    
    # Optimizer settings
    learning_rate=2e-4,
    optim="paged_adamw_8bit",  # Saves ~6GB VRAM
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    
    # Precision and memory
    bf16=True,  # Use bfloat16 mixed precision
    gradient_checkpointing=True,  # Reduces memory ~40%
    max_grad_norm=1.0,
    
    # Logging
    logging_steps=10,
    logging_dir=f"{output_dir}/logs",
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=250,
    
    # Checkpointing (critical for 16GB disk constraint)
    save_strategy="steps",
    save_steps=250,
    save_total_limit=3,  # Keep only 3 checkpoints (~300MB total)
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Other settings
    report_to="tensorboard",
    remove_unused_columns=False,
    seed=42,
)

print("Training configuration:")
print(f"  Output directory: {output_dir}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Max checkpoints: {training_args.save_total_limit}")

In [ ]:
# Initialize SFTTrainer
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

print("Initializing SFTTrainer...")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    
    # SFT-specific settings
    max_seq_length=2048,  # Limit context length
    packing=False,  # Disabled for reasoning tasks
    dataset_text_field="messages",  # Field containing chat messages
)

print("✓ SFTTrainer initialized and ready to train")
print(f"\nExpected memory usage: ~8-9GB VRAM (well under 32GB limit)")
print(f"Training steps per epoch: ~{len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

In [ ]:
# Start fine-tuning
print("Starting fine-tuning...\n")
print("This will take several hours depending on your GPU.")
print("Monitor progress in TensorBoard: %tensorboard --logdir ./deepseek-r1-math-science/logs\n")

# Train the model
trainer.train()

print("\n✓ Training complete!")

In [ ]:
# Save fine-tuned LoRA adapters
output_path = "./deepseek-r1-math-science-final"

print(f"Saving LoRA adapters to {output_path}...")
trainer.save_model(output_path)
tokenizer.save_pretrained(output_path)

print(f"\n✓ Model saved to {output_path}")
print(f"  LoRA adapter size: ~100MB")
print(f"\nYou can now use this model for inference or merge it with the base model.")

# Testing Fine-Tuned Model

Let's test the fine-tuned model on some math and science problems.

In [ ]:
# Test the fine-tuned model
from transformers import pipeline

print("Loading fine-tuned model for inference...")

# Create inference pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
)

# Test prompts
test_prompts = [
    "Solve for x: 3x + 5 = 20",
    "What is the derivative of f(x) = x^3 + 2x^2 - 5x + 3?",
    "Calculate the force required to accelerate a 10 kg object at 5 m/s^2.",
]

print("\nTesting fine-tuned model:\n")

for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    print("-" * 80)
    
    response = pipe(
        prompt,
        max_new_tokens=512,
        temperature=0.6,
        top_p=0.9,
        do_sample=True,
    )[0]["generated_text"]
    
    print(response)
    print("\n" + "=" * 80 + "\n")

# Optional: Merge LoRA Adapters with Base Model

For deployment, you can merge the LoRA adapters with the base model to create a standalone model.

In [ ]:
# Optional: Merge LoRA adapters with base model for deployment
print("Merging LoRA adapters with base model...")
print("This creates a standalone model (~3GB) that doesn't require PEFT.\n")

# Reload base model in fp16 (not quantized) for merging
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base model in fp16
merge_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# Load LoRA adapters
merge_model = PeftModel.from_pretrained(merge_model, output_path)

# Merge adapters into base model
merged_model = merge_model.merge_and_unload()

# Save merged model
merged_output_path = "./deepseek-r1-math-science-merged"
merged_model.save_pretrained(merged_output_path)
tokenizer.save_pretrained(merged_output_path)

print(f"\n✓ Merged model saved to {merged_output_path}")
print(f"  Size: ~3GB")
print(f"\nYou can now convert this to GGUF for on-device deployment using llama.cpp.")

# Visualise Attention

<p> Use TinyBERT & visualizew the self attention mechanism </p>

In [8]:
pip install -U bertviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.4 MB/s eta 0:00:00


In [10]:
from transformers import AutoModel
from bertviz import head_view, model_view

In [11]:
model=AutoModel.from_pretrained("prajjwal1/bert-tiny", output_attentions=True)
tokenizer=AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

prompt = "John's dog ate Nemis's cat"
inputs = tokenizer(prompt, return_tensors='pt')
with torch.no_grad():
  outputs = model(**inputs)



config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/39 [00:00<?, ?it/s]

BertModel LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.decoder.bias               | UNEXPECTED |  | 
cls.predictions.decoder.weight             | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
bert.embeddings.position_ids               | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

# Stop doing this blindly:
model = AutoModelForCausalLM.from_pretrained("model")

#### Start understanding:
#### - What are attention heads doing?
#### - How does positional encoding work?
#### - What's the difference between encoder-only (BERT), decoder-only (GPT), encoder-decoder (T5)?
#### - Why does model size scale quadratically with context length?

Action items:

Read "Attention Is All You Need" paper (seriously)
Implement a mini-transformer from scratch (100-200 lines)
Use model.generate(output_attentions=True) to visualize what the model "sees"
Study the HuggingFace modeling_*.py source code for your favorite models
Resources:

Andrej Karpathy's "Neural Networks: Zero to Hero" (YouTube)
Jay Alammar's "Illustrated Transformer" blog
HuggingFace Transformers source code

#### Bootcamp level:
model = AutoModelForCausalLM.from_pretrained("model", load_in_4bit=True)

#### Production level:
#### - Batching multiple requests efficiently
#### - KV-cache optimization for faster inference
#### - Flash Attention for 2-4x speedups
#### - ONNX/TensorRT conversion for production serving
#### - Model distillation to create smaller models
#### - Prompt caching to reduce redundant computation

Learn:

vLLM / Text Generation Inference (TGI): Production inference servers
GGML/llama.cpp: CPU-optimized inference
Triton Inference Server: Multi-model deployment
Batch processing: Processing 100 requests > processing 1 request 100 times
Continuous batching: Dynamic batching as requests arrive
Metrics that matter:

Throughput (tokens/second)
Latency (time to first token, time per token)
Cost per 1M tokens
GPU utilization %